# Department of Transportation - Oil Spill Mapping

In [41]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import openpyxl
import folium
import json
import gmaps.datasets
import numpy as np
import re
# from shapely.geometry import Point, Polygon
# import geopandas as gpd
# import descartes
import matplotlib.pyplot as plt
import plotly.graph_objects as go

%matplotlib inline
%matplotlib notebook

# Import API key
from config import g_map

# Configure gmaps
gmaps.configure(api_key=g_map)

## Bring in 2010 - Present Data

In [29]:
# 1. Read the Data_Columns_and_spills into a DataFrame. All sheets
# Unedited excel files
xlfile_2010 = pd.ExcelFile("../Resources/Data_columns_and_spills.xlsx", engine='openpyxl')
spill2010_df = xlfile_2010.parse('2010-Present') #'2010-Present' is the sheet name in the excel file
#spill2010_df is all data all columns
#spill2010_df.to_json('spill2010_df.json')



In [31]:
spill2010_df.columns

Index(['REPORT_RECEIVED_DATE', 'IYEAR', 'REPORT_NUMBER', 'SUPPLEMENTAL_NUMBER',
       'REPORT_TYPE', 'OPERATOR_ID', 'NAME', 'OPERATOR_STREET_ADDRESS',
       'OPERATOR_CITY_NAME', 'OPERATOR_STATE_ABBREVIATION',
       ...
       'PREPARER_TITLE', 'PREPARER_EMAIL', 'PREPARER_TELEPHONE',
       'PREPARER_FAX', 'PREPARED_DATE', 'AUTHORIZER_NAME', 'AUTHORIZER_TITLE',
       'AUTHORIZER_TELEPHONE', 'AUTHORIZER_EMAIL', 'NARRATIVE'],
      dtype='object', length=588)

In [33]:
spill2010_df

,REPORT_RECEIVED_DATE,IYEAR,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,REPORT_TYPE,OPERATOR_ID,NAME,OPERATOR_STREET_ADDRESS,OPERATOR_CITY_NAME,OPERATOR_STATE_ABBREVIATION,...,PREPARER_TITLE,PREPARER_EMAIL,PREPARER_TELEPHONE,PREPARER_FAX,PREPARED_DATE,AUTHORIZER_NAME,AUTHORIZER_TITLE,AUTHORIZER_TELEPHONE,AUTHORIZER_EMAIL,NARRATIVE
0,2018-12-10,2018,20180382,32252,SUPPLEMENTAL FINAL,31130,DCP MIDSTREAM,370 17TH STREET SUITE 2500,DENVER,CO,...,SR. PIPELINE COMPLIANCE SPECIALIST,kdneden@dcpmidstream.com,979-242-6976,NaN,2019-06-07,JOHN PONTIOUS,MANAGER PIPELINE COMPLIANCE,405-605-3859,jdpontious@dcpmidstream.com,DCP CONTRACT LINE LOCATOR GUADALUPE VALDES PRO...
1,2020-11-06,2020,20200300,34501,ORIGINAL FINAL,32545,"DENBURY GULF COAST PIPELINES, LLC",5851 LEGACY CIRCLE SUITE 1200,PLANO,TX,...,REGULATORY COMPLIANCE SPECIALIST,chad.docekal@denbury.com,972-673-2734,NaN,2020-11-06,DAVID SHEPPARD,SENIOR VICE PRESIDENT - OPERATIONS,972-673-2038,david.sheppard@denbury.com,"ON 10/7/2020 AT 17:31, DENBURY BEGAN BLOWDOWN ..."
2,2015-02-24,2015,20150071,21327,SUPPLEMENTAL FINAL,31618,ENTERPRISE PRODUCTS OPERATING LLC,1100 LOUISIANA STREET,HOUSTON,TX,...,SR. COMPLIANCE SPECIALIST,mjpavlak@eprod.com,713-381-5897,NaN,2016-04-21,MICHAEL PAVLAK,SR. COMPLIANCE SPECIALIST,713-381-5897,mjpavlak@eprod.com,AT APPROXIMATELY 9:40 A.M. C.S.T. ON JANUARY 2...
3,2020-09-13,2020,20200253,35187,SUPPLEMENTAL,2552,COLONIAL PIPELINE CO,1185 SANCTUARY PARKWAY SUITE 100,ALPHARETTA,GA,...,MANAGER - PIPELINE COMPLIANCE,mpiazza@colpipe.com,678-763-5911,NaN,2021-04-17,MARK PIAZZA,MANAGER - PIPELINE COMPLIANCE,678-763-5911,mpiazza@colpipe.com,"ON 8/14/2020 AT 18:20, A COLONIAL EMPLOYEE WAS..."
4,2015-02-06,2015,20150040,20630,SUPPLEMENTAL FINAL,31618,ENTERPRISE PRODUCTS OPERATING LLC,1100 LOUISIANA STREET,HOUSTON,TX,...,SR. PIPELINE COMPLIANCE SPECIALIST,wnburrell@eprod.com,713-381-3536,NaN,2015-07-29,NEAL BURRELL,SR. PIPELINE COMPLIANCE SPECIALIST,713-381-3536,wnburrell@eprod.com,"ON 1/10/2015 AT 20:45 HRS, PIPELINE CONTROL DI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,2018-01-08,2017,20180006,23089,SUPPLEMENTAL FINAL,3445,DIXIE PIPELINE COMPANY LLC,1100 LOUISIANA ST.,HOUSTON,TX,...,PIPELINE COMPLIANCE ENGINEER,lswilliams@eprod.com,713-381-6748,NaN,2018-02-01,SUZIE DAVIS,COMPLIANCE MANAGER,713-381-5585,smdavis@eprod.com,PUMP SHUT DOWN DUE TO HIGH BEARING TEMPERATURE...
4636,2018-05-30,2018,20180156,30565,SUPPLEMENTAL FINAL,30829,ENTERPRISE CRUDE PIPELINE LLC,1100 LOUISIANA STREET,HOUSTON,TX,...,PIPELINE COMPLIANCE SENIOR ENGINEER,lswilliams@eprod.com,713-381-6748,NaN,2018-06-28,LEANN WILLIAMS,PIPELINE COMPLIANCE SENIOR ENGINEER,713-381-6748,lswilliams@eprod.com,"DURING TANK REPAIRS, CONTRACT EMPLOYEE REMOVED..."
4637,2019-01-03,2018,20190001,31424,ORIGINAL FINAL,39145,ENBRIDGE STORAGE (CUSHING) L.L.C.,5400 WESTHEIMER COURT,HOUSTON,TX,...,SR COMPLIANCE ANALYST,stacy.soine@enbridge.com,715-395-3620,NaN,2019-01-03,RON CARLBERG,SUPERVISOR US PIPELINE COMPLIANCE,715-395-3619,ron.carlberg@enbridge.com,"ON DECEMBER 4, 2018 AT APPROXIMATELY 20:10 CST..."
4638,2019-01-24,2018,20190021,33804,SUPPLEMENTAL FINAL,300,"PLAINS PIPELINE, L.P.","333 CLAY STREET, SUITE 1900",HOUSTON,TX,...,ERC COMPLIANCE SPECIALIST,nagonzalez@paalp.com,713-646-4376,NaN,2020-05-27,NGIABI GICUHI,ERC MGR DIRECTOR,713-646-4336,ngicuhi@paalp.com,"ON 08/28/2018 AT ~7:40 A.M., THE PRIMARY AND S..."


## Mapping 2010-Present Spills

In [16]:
# Pulling useful columns for mapping purposes
my_columns = [
    'REPORT_NUMBER',
    'OPERATOR_ID',
    'LOCATION_LATITUDE',
    'LOCATION_LONGITUDE',
    'COMMODITY_RELEASED_TYPE',
    'UNINTENTIONAL_RELEASE_BBLS',
    'ON_OFF_SHORE',
    'ONSHORE_CITY_NAME',
    'ONSHORE_COUNTY_NAME',
    'ONSHORE_STATE_ABBREVIATION',
    'SYSTEM_PART_INVOLVED',
    'INSTALLATION_YEAR',
    'CAUSE'
    ]

In [17]:
# mapping columns called info2010_df
info2010_df = spill2010_df[my_columns]
# Creating another dataframe of first 100 rows called 'test' t2010_df
# Used to get map functioning and limiting API requests
t2010_df = info2010_df[:100]
t2010_df.to_json('test_df.json') # exports DataFrame to json for javascript and mapping functionality
t2010_df

,REPORT_NUMBER,OPERATOR_ID,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE_BBLS,ON_OFF_SHORE,ONSHORE_CITY_NAME,ONSHORE_COUNTY_NAME,ONSHORE_STATE_ABBREVIATION,SYSTEM_PART_INVOLVED,INSTALLATION_YEAR,CAUSE
0,20180382,31130,30.351556,-100.581792,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,48400.0,ONSHORE,Not Within a Municipality,SUTTON,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2013.0,EXCAVATION DAMAGE
1,20200300,32545,32.628233,-90.569264,CO2 (CARBON DIOXIDE),41177.0,ONSHORE,Not Within a Municipality,YAZOO COUNTY,MS,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2009.0,EQUIPMENT FAILURE
2,20150071,31618,40.366817,-80.584594,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,30565.0,ONSHORE,COLLIERS,BROOKE,WV,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2013.0,MATERIAL FAILURE OF PIPE OR WELD
3,20200253,2552,35.414106,-80.806185,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,28571.0,ONSHORE,HUNTERSVILLE,MECKLENBURG,NC,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1978.0,MATERIAL FAILURE OF PIPE OR WELD
4,20150040,31618,28.885881,-96.433027,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,27123.0,ONSHORE,Not Within a Municipality,JACKSON,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1966.0,MATERIAL FAILURE OF PIPE OR WELD
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,20100150,32296,30.259890,-93.209910,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,2237.0,ONSHORE,LAKE CHARLES,CALCASIEU,LA,"ONSHORE PIPELINE, INCLUDING VALVE SITES",NaN,NATURAL FORCE DAMAGE
96,20120009,22610,40.312770,-95.721250,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,2184.0,ONSHORE,NEMAHA CITY,NEMAH,NE,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1950.0,EXCAVATION DAMAGE
97,20100130,395,41.844019,-85.741131,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,2121.0,ONSHORE,CONSTANTINE,ST. JOSEPH,MI,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1953.0,MATERIAL FAILURE OF PIPE OR WELD
98,20170243,39504,30.085836,-97.386029,CRUDE OIL,2084.0,ONSHORE,Not Within a Municipality,BASTROP,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",NaN,EXCAVATION DAMAGE


In [69]:
# Join lat and long into one column...might be easier for mapping json
t2010_df.isnull().sum(axis = 0)

REPORT_NUMBER                 0
OPERATOR_ID                   0
LOCATION_LATITUDE             0
LOCATION_LONGITUDE            0
COMMODITY_RELEASED_TYPE       0
UNINTENTIONAL_RELEASE_BBLS    0
ON_OFF_SHORE                  0
ONSHORE_CITY_NAME             2
ONSHORE_COUNTY_NAME           1
ONSHORE_STATE_ABBREVIATION    0
SYSTEM_PART_INVOLVED          0
INSTALLATION_YEAR             7
CAUSE                         0
LatLong                       0
dtype: int64

In [70]:
## Attempting charts of incidents per year
spill2010_df

,REPORT_RECEIVED_DATE,IYEAR,REPORT_NUMBER,SUPPLEMENTAL_NUMBER,REPORT_TYPE,OPERATOR_ID,NAME,OPERATOR_STREET_ADDRESS,OPERATOR_CITY_NAME,OPERATOR_STATE_ABBREVIATION,...,PREPARER_TITLE,PREPARER_EMAIL,PREPARER_TELEPHONE,PREPARER_FAX,PREPARED_DATE,AUTHORIZER_NAME,AUTHORIZER_TITLE,AUTHORIZER_TELEPHONE,AUTHORIZER_EMAIL,NARRATIVE
0,2018-12-10,2018,20180382,32252,SUPPLEMENTAL FINAL,31130,DCP MIDSTREAM,370 17TH STREET SUITE 2500,DENVER,CO,...,SR. PIPELINE COMPLIANCE SPECIALIST,kdneden@dcpmidstream.com,979-242-6976,NaN,2019-06-07,JOHN PONTIOUS,MANAGER PIPELINE COMPLIANCE,405-605-3859,jdpontious@dcpmidstream.com,DCP CONTRACT LINE LOCATOR GUADALUPE VALDES PRO...
1,2020-11-06,2020,20200300,34501,ORIGINAL FINAL,32545,"DENBURY GULF COAST PIPELINES, LLC",5851 LEGACY CIRCLE SUITE 1200,PLANO,TX,...,REGULATORY COMPLIANCE SPECIALIST,chad.docekal@denbury.com,972-673-2734,NaN,2020-11-06,DAVID SHEPPARD,SENIOR VICE PRESIDENT - OPERATIONS,972-673-2038,david.sheppard@denbury.com,"ON 10/7/2020 AT 17:31, DENBURY BEGAN BLOWDOWN ..."
2,2015-02-24,2015,20150071,21327,SUPPLEMENTAL FINAL,31618,ENTERPRISE PRODUCTS OPERATING LLC,1100 LOUISIANA STREET,HOUSTON,TX,...,SR. COMPLIANCE SPECIALIST,mjpavlak@eprod.com,713-381-5897,NaN,2016-04-21,MICHAEL PAVLAK,SR. COMPLIANCE SPECIALIST,713-381-5897,mjpavlak@eprod.com,AT APPROXIMATELY 9:40 A.M. C.S.T. ON JANUARY 2...
3,2020-09-13,2020,20200253,35187,SUPPLEMENTAL,2552,COLONIAL PIPELINE CO,1185 SANCTUARY PARKWAY SUITE 100,ALPHARETTA,GA,...,MANAGER - PIPELINE COMPLIANCE,mpiazza@colpipe.com,678-763-5911,NaN,2021-04-17,MARK PIAZZA,MANAGER - PIPELINE COMPLIANCE,678-763-5911,mpiazza@colpipe.com,"ON 8/14/2020 AT 18:20, A COLONIAL EMPLOYEE WAS..."
4,2015-02-06,2015,20150040,20630,SUPPLEMENTAL FINAL,31618,ENTERPRISE PRODUCTS OPERATING LLC,1100 LOUISIANA STREET,HOUSTON,TX,...,SR. PIPELINE COMPLIANCE SPECIALIST,wnburrell@eprod.com,713-381-3536,NaN,2015-07-29,NEAL BURRELL,SR. PIPELINE COMPLIANCE SPECIALIST,713-381-3536,wnburrell@eprod.com,"ON 1/10/2015 AT 20:45 HRS, PIPELINE CONTROL DI..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,2018-01-08,2017,20180006,23089,SUPPLEMENTAL FINAL,3445,DIXIE PIPELINE COMPANY LLC,1100 LOUISIANA ST.,HOUSTON,TX,...,PIPELINE COMPLIANCE ENGINEER,lswilliams@eprod.com,713-381-6748,NaN,2018-02-01,SUZIE DAVIS,COMPLIANCE MANAGER,713-381-5585,smdavis@eprod.com,PUMP SHUT DOWN DUE TO HIGH BEARING TEMPERATURE...
4636,2018-05-30,2018,20180156,30565,SUPPLEMENTAL FINAL,30829,ENTERPRISE CRUDE PIPELINE LLC,1100 LOUISIANA STREET,HOUSTON,TX,...,PIPELINE COMPLIANCE SENIOR ENGINEER,lswilliams@eprod.com,713-381-6748,NaN,2018-06-28,LEANN WILLIAMS,PIPELINE COMPLIANCE SENIOR ENGINEER,713-381-6748,lswilliams@eprod.com,"DURING TANK REPAIRS, CONTRACT EMPLOYEE REMOVED..."
4637,2019-01-03,2018,20190001,31424,ORIGINAL FINAL,39145,ENBRIDGE STORAGE (CUSHING) L.L.C.,5400 WESTHEIMER COURT,HOUSTON,TX,...,SR COMPLIANCE ANALYST,stacy.soine@enbridge.com,715-395-3620,NaN,2019-01-03,RON CARLBERG,SUPERVISOR US PIPELINE COMPLIANCE,715-395-3619,ron.carlberg@enbridge.com,"ON DECEMBER 4, 2018 AT APPROXIMATELY 20:10 CST..."
4638,2019-01-24,2018,20190021,33804,SUPPLEMENTAL FINAL,300,"PLAINS PIPELINE, L.P.","333 CLAY STREET, SUITE 1900",HOUSTON,TX,...,ERC COMPLIANCE SPECIALIST,nagonzalez@paalp.com,713-646-4376,NaN,2020-05-27,NGIABI GICUHI,ERC MGR DIRECTOR,713-646-4336,ngicuhi@paalp.com,"ON 08/28/2018 AT ~7:40 A.M., THE PRIMARY AND S..."


## Mapping 2010-Present 'info2010_df' spill locations ~4000 locations

In [71]:
info_box_template = """
<dl>
<dt>Operator ID: </dt><dd>{OPERATOR_ID}</dd>
<dt>City</dt><dd>{ONSHORE_CITY_NAME}</dd>
<dt>State</dt><dd>{ONSHORE_STATE_ABBREVIATION}</dd>
<dt>Spill Commodity</dt><dd>{COMMODITY_RELEASED_TYPE}</dd>
<dt>Spill Volume (bbls)</dt><dd>{UNINTENTIONAL_RELEASE_BBLS}</dd>
</dl>

"""


spill_info = [info_box_template.format(**row) for index, row in info2010_df.iterrows()]

In [72]:
# Create map from lat and long
locations = info2010_df[["LOCATION_LATITUDE","LOCATION_LONGITUDE"]]
spill = info2010_df["UNINTENTIONAL_RELEASE_BBLS"]
fig = gmaps.figure(center=(36, -97), zoom_level=4)
heat_layer = gmaps.heatmap_layer(locations, weights=spill,dissipating=False,
            max_intensity=30000, point_radius=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=spill_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

## Add another map 
### Filtering only pipeline, corrosion failures
### showing property damage instead of spill volume

In [8]:
my_columns = [
    'REPORT_NUMBER',
    'OPERATOR_ID',
    'LOCATION_LATITUDE',
    'LOCATION_LONGITUDE',
    'COMMODITY_RELEASED_TYPE',
    'UNINTENTIONAL_RELEASE_BBLS',
    'ON_OFF_SHORE',
    'ONSHORE_CITY_NAME',
    'ONSHORE_COUNTY_NAME',
    'ONSHORE_STATE_ABBREVIATION',
    'SYSTEM_PART_INVOLVED',
    'INSTALLATION_YEAR',
    'CAUSE',
    'PRPTY'
    ]

In [40]:
pipeline_df = spill2010_df[my_columns]
pipeline_df

,REPORT_NUMBER,OPERATOR_ID,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE_BBLS,ON_OFF_SHORE,ONSHORE_CITY_NAME,ONSHORE_COUNTY_NAME,ONSHORE_STATE_ABBREVIATION,SYSTEM_PART_INVOLVED,INSTALLATION_YEAR,CAUSE,PRPTY
0,20180382,31130,30.351556,-100.581792,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,48400.0,ONSHORE,Not Within a Municipality,SUTTON,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2013.0,EXCAVATION DAMAGE,15200000.0
1,20200300,32545,32.628233,-90.569264,CO2 (CARBON DIOXIDE),41177.0,ONSHORE,Not Within a Municipality,YAZOO COUNTY,MS,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2009.0,EQUIPMENT FAILURE,64998.0
2,20150071,31618,40.366817,-80.584594,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,30565.0,ONSHORE,COLLIERS,BROOKE,WV,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2013.0,MATERIAL FAILURE OF PIPE OR WELD,6910591.0
3,20200253,2552,35.414106,-80.806185,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,28571.0,ONSHORE,HUNTERSVILLE,MECKLENBURG,NC,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1978.0,MATERIAL FAILURE OF PIPE OR WELD,64253139.0
4,20150040,31618,28.885881,-96.433027,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,27123.0,ONSHORE,Not Within a Municipality,JACKSON,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1966.0,MATERIAL FAILURE OF PIPE OR WELD,782260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,20180006,3445,33.362982,-83.447904,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,0.0,ONSHORE,EATONTON,PUTNAM,GA,ONSHORE PUMP/METER STATION EQUIPMENT AND PIPING,NaN,EQUIPMENT FAILURE,8588.0
4636,20180156,30829,29.741758,-95.124244,CRUDE OIL,0.0,ONSHORE,HOUSTON,HARRIS,TX,"ONSHORE BREAKOUT TANK OR STORAGE VESSEL, INCLU...",NaN,INCORRECT OPERATION,0.0
4637,20190001,39145,35.940630,-96.748440,CRUDE OIL,0.0,ONSHORE,CUSHING,LINCOLN,OK,ONSHORE TERMINAL/TANK FARM EQUIPMENT AND PIPING,2018.0,EQUIPMENT FAILURE,8368.0
4638,20190021,300,33.916010,-98.436540,CRUDE OIL,0.0,ONSHORE,WICHITA FALLS,WICHITA,TX,"ONSHORE BREAKOUT TANK OR STORAGE VESSEL, INCLU...",1949.0,INCORRECT OPERATION,2030538.0


In [41]:
# # changing formatting of Property column to currency
# pipeline_df['PRPTY'] = pipeline_df['PRPTY'].apply(lambda x: "${:,.0f}".format(x))
# pipeline_df

In [42]:
# Filtering causes; showing only corrosion failure, material failure of pipe or weld and equipment failure.
pipeline_df.loc[(pipeline_df["CAUSE"] == "CORROSION FAILURE") & 
                       (pipeline_df["CAUSE"] == "MATERIAL FAILURE OF PIPE OR WELD") &
                       (pipeline_df["CAUSE"] == "EQUIPMENT FAILURE")]
pipeline_df

,REPORT_NUMBER,OPERATOR_ID,LOCATION_LATITUDE,LOCATION_LONGITUDE,COMMODITY_RELEASED_TYPE,UNINTENTIONAL_RELEASE_BBLS,ON_OFF_SHORE,ONSHORE_CITY_NAME,ONSHORE_COUNTY_NAME,ONSHORE_STATE_ABBREVIATION,SYSTEM_PART_INVOLVED,INSTALLATION_YEAR,CAUSE,PRPTY
0,20180382,31130,30.351556,-100.581792,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,48400.0,ONSHORE,Not Within a Municipality,SUTTON,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2013.0,EXCAVATION DAMAGE,15200000.0
1,20200300,32545,32.628233,-90.569264,CO2 (CARBON DIOXIDE),41177.0,ONSHORE,Not Within a Municipality,YAZOO COUNTY,MS,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2009.0,EQUIPMENT FAILURE,64998.0
2,20150071,31618,40.366817,-80.584594,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,30565.0,ONSHORE,COLLIERS,BROOKE,WV,"ONSHORE PIPELINE, INCLUDING VALVE SITES",2013.0,MATERIAL FAILURE OF PIPE OR WELD,6910591.0
3,20200253,2552,35.414106,-80.806185,REFINED AND/OR PETROLEUM PRODUCT (NON-HVL) WHI...,28571.0,ONSHORE,HUNTERSVILLE,MECKLENBURG,NC,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1978.0,MATERIAL FAILURE OF PIPE OR WELD,64253139.0
4,20150040,31618,28.885881,-96.433027,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,27123.0,ONSHORE,Not Within a Municipality,JACKSON,TX,"ONSHORE PIPELINE, INCLUDING VALVE SITES",1966.0,MATERIAL FAILURE OF PIPE OR WELD,782260.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4635,20180006,3445,33.362982,-83.447904,HVL OR OTHER FLAMMABLE OR TOXIC FLUID WHICH IS...,0.0,ONSHORE,EATONTON,PUTNAM,GA,ONSHORE PUMP/METER STATION EQUIPMENT AND PIPING,NaN,EQUIPMENT FAILURE,8588.0
4636,20180156,30829,29.741758,-95.124244,CRUDE OIL,0.0,ONSHORE,HOUSTON,HARRIS,TX,"ONSHORE BREAKOUT TANK OR STORAGE VESSEL, INCLU...",NaN,INCORRECT OPERATION,0.0
4637,20190001,39145,35.940630,-96.748440,CRUDE OIL,0.0,ONSHORE,CUSHING,LINCOLN,OK,ONSHORE TERMINAL/TANK FARM EQUIPMENT AND PIPING,2018.0,EQUIPMENT FAILURE,8368.0
4638,20190021,300,33.916010,-98.436540,CRUDE OIL,0.0,ONSHORE,WICHITA FALLS,WICHITA,TX,"ONSHORE BREAKOUT TANK OR STORAGE VESSEL, INCLU...",1949.0,INCORRECT OPERATION,2030538.0


In [ ]:
info_box_template = """
<dl>
<dt>Operator ID: </dt><dd>{OPERATOR_ID}</dd>
<dt>City</dt><dd>{ONSHORE_CITY_NAME}</dd>
<dt>State</dt><dd>{ONSHORE_STATE_ABBREVIATION}</dd>
<dt>Cause:</dt><dd>{CAUSE}</dd>
<dt>Spill Commodity</dt><dd>{COMMODITY_RELEASED_TYPE}</dd>
<dt>Spill Volume (bbls)</dt><dd>{UNINTENTIONAL_RELEASE_BBLS}</dd>
<dt>Spill Cost</dt><dd>{PRPTY}</dd>
</dl>

"""

# Using the pipeline_df to gather spill info for marker data...
spill_info = [info_box_template.format(**row) for index, row in pipeline_df.iterrows()]

In [ ]:
# Create map from lat and long
locations = pipeline_df[["LOCATION_LATITUDE","LOCATION_LONGITUDE"]]
spill = pipeline_df["PRPTY"]
fig = gmaps.figure(center=(36, -97), zoom_level=4)
heat_layer = gmaps.heatmap_layer(locations, weights=spill,dissipating=False,
            max_intensity=1000000000, point_radius=2)
marker_layer = gmaps.marker_layer(locations, info_box_content=spill_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
#Display the figure
fig